In [22]:
## The following packages were recommended based on the article found at https://towardsdatascience.com/clustering-geospatial-data-f0584f0b04ec
## for data
import numpy as np
import pandas as pd
## for plotting
import matplotlib.pyplot as plt
import seaborn as sns
## for geospatial
import folium
import geopy
## for machine learning
from sklearn import preprocessing, cluster
import scipy
## for deep learning
import minisom

## for binary and os operations
import glob
import os

In [23]:
all_csvs = glob.glob(os.path.join('.', '*_us.csv'))
epa_tri = pd.concat((pd.read_csv(f) for f in all_csvs), ignore_index=True)

/tmp/ipykernel_552/2000873867.py:2: DtypeWarning: Columns (15,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  epa_tri = pd.concat((pd.read_csv(f) for f in all_csvs), ignore_index=True)
/tmp/ipykernel_552/2000873867.py:2: DtypeWarning: Columns (8,10,21,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  epa_tri = pd.concat((pd.read_csv(f) for f in all_csvs), ignore_index=True)
/tmp/ipykernel_552/2000873867.py:2: DtypeWarning: Columns (15,22,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  epa_tri = pd.concat((pd.read_csv(f) for f in all_csvs), ignore_index=True)
/tmp/ipykernel_552/2000873867.py:2: DtypeWarning: Columns (23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  epa_tri = pd.concat((pd.read_csv(f) for f in all_csvs), ignore_index=True)
/tmp/ipykernel_552/2000873867.py:2: DtypeWarning: Columns (8,21,23,25) have mixed types. Specify dtype option on im

In [27]:
epa_tri.head()

,1. YEAR,2. TRIFD,3. FRS ID,4. FACILITY NAME,5. STREET ADDRESS,6. CITY,7. COUNTY,8. ST,9. ZIP,10. BIA,...,110. 8.2 - ENERGY RECOVER ON,111. 8.3 - ENERGY RECOVER OF,112. 8.4 - RECYCLING ON SITE,113. 8.5 - RECYCLING OFF SIT,114. 8.6 - TREATMENT ON SITE,115. 8.7 - TREATMENT OFF SITE,116. PRODUCTION WSTE (8.1-8.7),117. 8.8 - ONE-TIME RELEASE,118. PROD_RATIO_OR_ ACTIVITY,119. 8.9 - PRODUCTION RATIO
0,1993,07008STFLXMIDDL,110000318040,STAFLEX PRODS,50 MIDDLESEX AVENUE,CARTERET,MIDDLESEX,NJ,7008,NaN,...,0.0,46674.0,0.0,0.0,0.0,132340.0,179014.0,0.0,NaN,1.07
1,1993,28658MRDTHSTJAM,110000587017,RR DONNELLEY PRINTING CO,ST JAMES CHURCH RD,NEWTON,CATAWBA,NC,286580289,NaN,...,0.0,0.0,0.0,32000.0,0.0,190.0,32190.0,0.0,NaN,0.81
2,1993,91749THDXT15051,110000495571,HENKEL CORP,15051 E DON JULIAN RD,CITY OF INDUSTRY,LOS ANGELES,CA,91746,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.83
3,1993,32206FMCCR1200T,110000361885,FMC CORP,1200 TALLEYRAND AVE,JACKSONVILLE,DUVAL,FL,32206,NaN,...,0.0,0.0,0.0,0.0,0.0,303.0,303.0,0.0,NaN,0.98
4,1993,91352NPLCR11912,110000476994,NUPLA CORP LLC,11912 SHELDON ST,SUN VALLEY,LOS ANGELES,CA,91352,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.00


In [24]:
fda_cdi = pd.read_csv('CDI_with_pop_centers.csv')
fda_cdi.head()

,YearStart,YearEnd,LocationAbbr,LocationDesc,DataSource,Topic,Question,Response,DataValueUnit,DataValueType,...,StratificationCategoryID1,StratificationID1,StratificationCategoryID2,StratificationID2,StratificationCategoryID3,StratificationID3,Population,Latitude,Longitude,Centroid
0,2015,2015,MS,Mississippi,NVSS,Alcohol,Chronic liver disease mortality,NaN,NaN,Number,...,OVERALL,OVR,NaN,NaN,NaN,NaN,2844658,32.56642,-89.593164,"{""type"":""Point"", ""coordinates"":[-89.593164,32...."
1,2014,2014,MS,Mississippi,NVSS,Alcohol,Chronic liver disease mortality,NaN,"cases per 100,000",Age-adjusted Rate,...,OVERALL,OVR,NaN,NaN,NaN,NaN,2844658,32.56642,-89.593164,"{""type"":""Point"", ""coordinates"":[-89.593164,32...."
2,2016,2016,MS,Mississippi,NVSS,Alcohol,Chronic liver disease mortality,NaN,"cases per 100,000",Crude Rate,...,OVERALL,OVR,NaN,NaN,NaN,NaN,2844658,32.56642,-89.593164,"{""type"":""Point"", ""coordinates"":[-89.593164,32...."
3,2014,2014,MS,Mississippi,NVSS,Alcohol,Chronic liver disease mortality,NaN,"cases per 100,000",Crude Rate,...,OVERALL,OVR,NaN,NaN,NaN,NaN,2844658,32.56642,-89.593164,"{""type"":""Point"", ""coordinates"":[-89.593164,32...."
4,2017,2017,MS,Mississippi,NVSS,Alcohol,Chronic liver disease mortality,NaN,NaN,Number,...,OVERALL,OVR,NaN,NaN,NaN,NaN,2844658,32.56642,-89.593164,"{""type"":""Point"", ""coordinates"":[-89.593164,32...."


In [29]:
plant_locations = epa_tri[['1. YEAR', '2. TRIFD', '4. FACILITY NAME', '8. ST', '12. LATITUDE', '13. LONGITUDE']]
plant_locations.head()

,1. YEAR,2. TRIFD,4. FACILITY NAME,8. ST,12. LATITUDE,13. LONGITUDE
0,1993,07008STFLXMIDDL,STAFLEX PRODS,NJ,40.564583,-74.224278
1,1993,28658MRDTHSTJAM,RR DONNELLEY PRINTING CO,NC,35.640822,-81.207941
2,1993,91749THDXT15051,HENKEL CORP,CA,34.023633,-117.972636
3,1993,32206FMCCR1200T,FMC CORP,FL,30.340102,-81.631472
4,1993,91352NPLCR11912,NUPLA CORP LLC,CA,34.240360,-118.392280
